<a href="https://colab.research.google.com/github/xinnazim/esaa/blob/main/%ED%95%AD%EA%B3%B5%ED%8E%B8_%EC%A7%80%EC%97%B0_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 데이터 및 라이브러리 로딩**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') #matplotlib 한글 깨짐 방지

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 데이터 불러오기


train=pd.read_csv("/content/drive/MyDrive/air/train.csv")
test = pd.read_csv("/content/drive/MyDrive/air/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/air/sample_submission.csv") # 제출용 파일

# **1. EDA**

In [ ]:
train.head()

In [8]:
train.tail()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN
999999,TRAIN_999999,3,26,1800.0,2056.0,0,0,RSW,14635,Florida,DTW,11433,Michigan,1084.0,NaN,DL,19790.0,N695DL,NaN


In [9]:
test.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number
0,TEST_000000,12,16,1156.0,NaN,0,0,IAH,12266,Texas,SAT,14683,Texas,191.0,United Air Lines Inc.,UA,NaN,N79402
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,ATL,10397,NaN,746.0,Delta Air Lines Inc.,DL,19790.0,N3765
2,TEST_000002,3,6,1600.0,1915.0,0,0,ORD,13930,Illinois,LGA,12953,New York,733.0,United Air Lines Inc.,UA,19977.0,N413UA
3,TEST_000003,5,18,1920.0,2045.0,0,0,OAK,13796,California,LAX,12892,California,337.0,Southwest Airlines Co.,WN,19393.0,N905WN
4,TEST_000004,7,7,1915.0,2152.0,0,0,FLL,11697,Florida,LAX,12892,California,2343.0,JetBlue Airways,B6,20409.0,N945JT


In [10]:
submission.head()

,ID,Not_Delayed,Delayed
0,TEST_000000,0,1
1,TEST_000001,0,1
2,TEST_000002,0,1
3,TEST_000003,0,1
4,TEST_000004,0,1


In [11]:
print(train.shape)
print(test.shape)
print(submission.shape)

(1000000, 19)
(1000000, 18)
(1000000, 3)


* ID : 샘플 고유 id
* Month: 해당 항공편의 출발 월
* Day_of_Month: Month에 해당하는 월의 날짜
* Estimated_Departure_Time: 전산 시스템을 바탕으로 측정된 비행기의 출발 시간 (현지 시각, HH:MM 형식)
* Estimated_Arrival_Time: 전산 시스템을 바탕으로 측정된 비행기의 도착 시간 (현지 시각, HH:MM 형식)
* Cancelled: 해당 항공편의 취소 여부 (0: 취소되지 않음, 1: 취소됨)
* Diverted: 해당 항공편의 경유 여부 (0: 취소되지 않음, 1: 취소됨)
* Origin_Airport: 해당 항공편 출발 공항의 고유 코드 (IATA 공항 코드)
* Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
* Origin_State: 해당 항공편 출발 공항이 위치한 주의 이름
* Destination_Airport: 해당 항공편 도착 공항의 고유 코드 (IATA 공항 코드)
* Destination_Airport_ID: 해당 항공편 도착 공항의 고유 ID (US DOT ID)
* Destination_State: 해당 항공편 도착 공항이 위치한 주의 이름
* Distance: 출발 공항과 도착 공항 사이의 거리 (mile 단위)
* Airline: 해당 항공편을 운항하는 항공사
* Carrier_Code(IATA): 해당 항공편을 운항하는 항공사의 고유 코드
(IATA 공항 코드, 단 다른 항공사가 같은 코드를 보유할 수도 있음)
* Carrier_ID(DOT): 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
* Tail_Number: 해당 항공편을 운항하는 항공기의 고유 등록번호
* Delay: 항공편 지연 여부 (Not_Delayed, Delayed)

In [7]:
train.info()
# null: Estimated_Departure_Time, Estimated_Arrival_Time, Origin_State ,Destination_State, Airline, Carrier_Code(IATA), Carrier_ID(DOT)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              890985 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         890921 non-null   object 
 13  Distance                  10

In [13]:
pd.options.display.float_format = '{:.1f}'.format # 소숫점 자리수 출력값 format 설정
train.describe()

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT)
count,1000000.0,1000000.0,890981.0,890960.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,891003.0
mean,6.9,15.8,1341.2,1493.3,0.0,0.0,12696.3,12701.8,784.1,19997.4
std,3.5,8.8,489.8,520.8,0.0,0.0,1514.9,1515.2,590.8,404.3
min,1.0,1.0,1.0,1.0,0.0,0.0,10135.0,10135.0,16.0,19393.0
25%,4.0,8.0,925.0,1105.0,0.0,0.0,11292.0,11292.0,350.0,19790.0
50%,7.0,16.0,1332.0,1524.0,0.0,0.0,12889.0,12889.0,623.0,19977.0
75%,10.0,23.0,1742.0,1924.0,0.0,0.0,14057.0,14057.0,1020.0,20378.0
max,12.0,31.0,2359.0,2400.0,0.0,0.0,16869.0,16869.0,5095.0,21171.0


In [14]:
train.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

In [15]:
test.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    108984
Estimated_Arrival_Time      109048
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                106505
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           106523
Distance                         0
Airline                     106527
Carrier_Code(IATA)          108993
Carrier_ID(DOT)             109006
Tail_Number                      0
dtype: int64

* iD : 샘플 고유 id
* Month: 해당 항공편의 출발 월(날씨에 영향을 받을거라 생각)
* Estimated_Departure_Time: null o, 비행기의 출발 시간 (현지 시각, HH:MM 형식)
* Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID) 코드 대신 ID 사용
* Destination_Airport_ID: 해당 항공편 도착 공항의 고유 ID (US DOT ID)
* Distance: 출발 공항과 도착 공항 사이의 거리 (mile 단위)
* Airline: null o, 해당 항공편을 운항하는 항공사
* Carrier_ID(DOT): null o, 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
* Tail_Number: 해당 항공편을 운항하는 항공기의 고유 등록번호
* Delay: 항공편 지연 여부 (Not_Delayed, Delayed)

# **2. 데이터 전처리**

In [18]:
# airline, carrier_ID 결측치 채우기
cond1 = train['Carrier_ID(DOT)'].isnull()
cond2 = ~train['Airline'].isnull()
print("Carrier_ID(DOT) 복구 가능한 데이터의 개수 :", len(train.loc[cond1 & cond2, :]))

Carrier_ID(DOT) 복구 가능한 데이터의 개수 : 97114


In [19]:
# airline to carrier id, dictinary 만들기
# 모두 데이터가 존재하는 열에서 Dict[Airline] = carrier_ID(DOT) 가 되도록 dictionary 생성
airline_to_cid = {}
for _, row in train[(~train['Carrier_ID(DOT)'].isnull() & ~train['Airline'].isnull())].iterrows():
    airline_to_cid[row['Airline']] = row['Carrier_ID(DOT)']

In [20]:
# carrier_ID 복구하기
def to_cid(x):
    return airline_to_cid[x]

cond1 = train['Carrier_ID(DOT)'].isnull()
cond2 = ~train['Airline'].isnull()
train.loc[cond1&cond2, 'Carrier_ID(DOT)'] = train.loc[cond1&cond2, 'Airline'].apply(lambda x: to_cid(x))

In [26]:
# 복구 안 된 Carrier_ID 제거
train = train.dropna(subset=['Carrier_ID(DOT)'], how='any', axis=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988117 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        988117 non-null  object 
 1   Month                     988117 non-null  int64  
 2   Day_of_Month              988117 non-null  int64  
 3   Estimated_Departure_Time  880391 non-null  float64
 4   Estimated_Arrival_Time    880328 non-null  float64
 5   Cancelled                 988117 non-null  int64  
 6   Diverted                  988117 non-null  int64  
 7   Origin_Airport            988117 non-null  object 
 8   Origin_Airport_ID         988117 non-null  int64  
 9   Origin_State              880378 non-null  object 
 10  Destination_Airport       988117 non-null  object 
 11  Destination_Airport_ID    988117 non-null  int64  
 12  Destination_State         880335 non-null  object 
 13  Distance                  988117 non-null  f

In [24]:
# test는 복구가 안된 row도 예측에 필요하기 때문에 airline, carrier_ID가 모두 없는 row는 최빈값으로 채움
# Airline, Carrier_Code 둘 다 없으면 최빈 값으로 대체
NaN_col = ['Carrier_ID(DOT)']
cond1 = test['Airline'].isnull()
cond2 = test['Carrier_ID(DOT)'].isnull()

for col in NaN_col:
    mode = test[col].mode()[0]
    test.loc[cond1&cond2, col] = mode

# 나머진 Airline에서 대체
cond1 = test['Carrier_ID(DOT)'].isnull()
cond2 = ~test['Airline'].isnull()
test.loc[cond1&cond2, 'Carrier_ID(DOT)'] = test.loc[cond1&cond2, 'Airline'].apply(lambda x: to_cid(x))


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  891016 non-null   float64
 4   Estimated_Arrival_Time    890952 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              893495 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         893477 non-null   object 
 13  Distance                  10

In [28]:
# 필요없는 columns 제거

col_drop = ['Day_of_Month', 'Cancelled', 'Diverted', 'Origin_Airport', 'Destination_Airport', 'Carrier_Code(IATA)', 'Airline', 'Origin_State', 'Destination_State']
train = train.drop(col_drop, axis=1)
test = test.drop(col_drop, axis=1)

Drop Done.


In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988117 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        988117 non-null  object 
 1   Month                     988117 non-null  int64  
 2   Estimated_Departure_Time  880391 non-null  float64
 3   Estimated_Arrival_Time    880328 non-null  float64
 4   Origin_Airport_ID         988117 non-null  int64  
 5   Destination_Airport_ID    988117 non-null  int64  
 6   Distance                  988117 non-null  float64
 7   Carrier_ID(DOT)           988117 non-null  float64
 8   Tail_Number               988117 non-null  object 
 9   Delay                     252016 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 82.9+ MB
